In [1]:
import numpy as np
import keras
from matplotlib.pyplot import imshow
import h5py
import cv2
from zipfile import ZipFile
keras.backend.set_image_data_format('channels_last')
from keras.layers import Input, Activation, Conv2D
from keras.models import Model

Using TensorFlow backend.


In [2]:
zipped_images = ZipFile('train_images.zip')

In [ ]:
# 查看目录结构
zipped_images.printdir()

In [3]:
# 提取图片
data = [] # 图片数据
lens = [] # 三种图片的数量
label = [] # 标签
mid_name = ['off_edge', 'on_edge', 'on_corner']

In [4]:
for i in range(3):
    txt_file_name = 'train_images/edge/' + mid_name[i] + '.txt' # 各个图片名在.txt文件里
    txtstr = zipped_images.read(txt_file_name) # 读出byte流
    print(type(txtstr))
    txtlist = txtstr.split() # 利用换行符分割
    # print(type(image_name)), type(image_name) = bytes
    for image_name in txtlist:
        # image_name是utf-8编码，str是unicode编码，要转换一下
        bmpname = 'train_images/edge/' + mid_name[i] + '/4/' + str(image_name, encoding='utf-8') 
        # 读出byte流
        bmp_byte_str = zipped_images.read(bmpname)
        # 转换为numpy array
        bmp_byte_array = np.frombuffer(bmp_byte_str, dtype=np.uint8)
        # 转换为灰度图像的格式
        image = cv2.imdecode(bmp_byte_array, cv2.IMREAD_GRAYSCALE)
        data.append(image)
    lens.append(len(txtlist))
    label.extend([i] * lens[-1])

<class 'bytes'>
<class 'bytes'>
<class 'bytes'>


In [5]:
print(data[110000])

[[  0   0   0   0   0]
 [  0   0   0   0   0]
 [191 191 191 191 191]
 [191 191 191 191 191]
 [191 191 191 191 191]]


In [6]:
print(label[110000])

1


In [7]:
print(lens)

[100390, 110910, 47060]


In [8]:
data = np.array(data)
label = np.array(label)

In [9]:
data = data.reshape((data.shape[0], -1))
label = label.reshape((label.shape[0], -1))

In [10]:
print(data.shape, label.shape)

(258360, 25) (258360, 1)


In [12]:
overall_data = np.hstack((data, label)) # 横向堆叠

In [13]:
overall_data = overall_data[0:100390 + 110910] # 取出on edge 和 off edge部分

In [14]:
print(overall_data.shape)

(211300, 26)


In [15]:
np.random.shuffle(overall_data)

In [16]:
div_num = int(overall_data.shape[0] * 0.7)
print(div_num)

147910


In [17]:
# 划分训练集和测试集
X_train = overall_data[0:div_num, 0:25]
Y_train = overall_data[0:div_num, 25]
X_test = overall_data[div_num:, 0:25]
Y_test = overall_data[div_num:, 25]



In [18]:
# 归一化
X_train = X_train / 255
X_test = X_test / 255

# 构造channel维度
X_train = X_train.reshape(X_train.shape[0], 5, 5, 1)
Y_train = Y_train.reshape(Y_train.shape[0], 1, 1, 1)
X_test = X_test.reshape(X_test.shape[0], 5, 5, 1)
Y_test = Y_test.reshape(Y_test.shape[0], 1, 1, 1)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(147910, 5, 5, 1) (147910, 1, 1, 1) (63390, 5, 5, 1) (63390, 1, 1, 1)


In [19]:
# 定义DNN网络结构
# DNN的好处在于，对于不同大小的输入，无需改变网络结构

def EdgeDetectModel(input_shape):

    X_input = Input(input_shape) # (5, 5) / (240, 240)
    
    # Conv2D(filter_number, filter_size, strides, name) 
    X = Conv2D(8, (3, 3), strides = (1, 1), name = 'conv0')(X_input) # (3, 3, 8) / (238, 238, 8)

    X = Activation('relu')(X)

    X = Conv2D(1, (3, 3), strides = (1, 1), name = 'conv1')(X) # (1, 1, 1) / (236, 236, 1)

    X = Activation('relu')(X)
    
    model = Model(inputs = X_input, outputs = X, name='EdgeDetectModel')

    return model

In [20]:
# None表示图片尺寸不确定
model = EdgeDetectModel((None, None, 1))

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# 确定优化器，损失函数，
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# 开始训练
model.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 16)
# 用test集，检测效果
preds = model.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [21]:
def load_test_images():
    data = []

    pre_path = './synthetic_characters/'
    for i in range(4):
        for j in range(10):
            image_path = pre_path + '0-0-0-' + str(i) + '-' + str(j) + '.bmp'
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            data.append(img)

    data = np.array(data)
    data = np.expand_dims(data, axis=3)
    return data
test_images = load_test_images()

In [22]:
print(test_images.shape)

(40, 240, 240, 1)


In [ ]:
import matplotlib.pyplot as plt
for i in range(test_images.shape[0]):
    test = test_images[i]
    test = np.expand_dims(test, axis=0)
    pred = model.predict(test)
    # print(pred.shape)
    pred = pred.reshape((236, 236))
    pred = (pred > 0.5)
    pred = pred * 255
    # plt.imshow(pred, cmap='gray')
    # plt.show()
    plt.imsave('edge_detection_test/' + str(i) + '.png', pred, cmap='gray')
